In [1]:
from cameo import load_model
from os import getcwd
from Functions_Modules import curation_tools as ct
from math import isnan
from numpy import zeros

/Users/clie/.virtualenvs/modeling/lib/python2.7/site-packages/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")


In [2]:
relative_directory = getcwd()
model = load_model(relative_directory + '/Reconstructions/MethylococcusModel13.json')

In [3]:
from cameo import Reaction,Metabolite

In [4]:
rxn = Reaction('NIT_mc')
rxn.add_metabolites({model.metabolites.atp_c:-16,
                    model.metabolites.h2o_c:-16,
                    model.metabolites.n2_c:-1,
                    model.metabolites.fdxrd_c:-4,
                    model.metabolites.adp_c:16,
                    model.metabolites.pi_c:16,
                    model.metabolites.fdxox_c:4,
                    model.metabolites.h_c:14, 
                    model.metabolites.h2_c:1,
                    model.metabolites.nh4_c:2},combine=False)
rxn.name = 'Nitrogenase'
rxn.id = 'NIT_mc'
rxn.annotation = {
 u'biocyc': u'NITROGENASE-RXN',
 u'brenda': u'1.18.6.1',
 u'kegg.reaction': u'R05185',
 u'metanetx.reaction': u'MNXR85317'}
rxn.gene_reaction_rule = 'MCA0229 and MCA0230 and MCA0231'

model.add_reaction(rxn)

In [5]:
model.reactions.NAD_H2.change_bounds(-1000,1000)

In [6]:
model.metabolites.h2_c.reactions

frozenset({<Reaction NIT_mc at 0x11a35e8d0>, <Reaction NAD_H2 at 0x11a5d6050>})

In [7]:
n2_p = model.metabolites.n2_c.copy()
n2_p.compartment = 'p'
n2_p.id = 'n2_p'
model.add_metabolite(n2_p)

In [8]:
n2_e = model.metabolites.n2_c.copy()
n2_e.compartment = 'e'
n2_e.id = 'n2_e'
model.add_metabolite(n2_e)

In [9]:
rxn = Reaction('N2trpp')
rxn.add_metabolites({model.metabolites.n2_c:1,
                    model.metabolites.n2_p:-1})
rxn.name = 'Nitrogen exchange, diffusion (periplasm)'
rxn.id = 'N2trpp'
rxn.annotation = {u'metanetx.reaction': u'MNXR3686'}

model.add_reaction(rxn)

In [10]:
rxn = Reaction('N2tex')
rxn.add_metabolites({model.metabolites.n2_p:1,
                    model.metabolites.n2_e:-1})
rxn.name = 'Nitrogen exchange, diffusion (extra-organism to periplasm)'
rxn.id = 'N2tex'
rxn.annotation = {u'metanetx.reaction': u'MNXR3686'}

model.add_reaction(rxn)

In [11]:
rxn = Reaction('EX_n2_e')
rxn.add_metabolites({model.metabolites.n2_e:-1})
rxn.name = 'N2 exchange'
rxn.id = 'EX_n2_e'
rxn.annotation = {u'metanetx.reaction': u'MNXR3686'}

model.add_reaction(rxn)

In [12]:
model.reactions.N2tex.change_bounds(-1000,1000)
model.reactions.N2trpp.change_bounds(-1000,1000)

In [13]:
model.reactions.EX_no3_e.lower_bound = 0

In [14]:
model.reactions.EX_nh4_e.lower_bound = 0

In [15]:
model.reactions.EX_n2_e.lower_bound = -1000

In [16]:
model.solve().f

0.0879773465210888

In [17]:
import cobra

target_filename_json = relative_directory + '/Reconstructions/MethylococcusModel14.json'
target_filename_xml = relative_directory + '/Reconstructions/MethylococcusModel14.xml'
cobra.io.write_sbml_model(model, target_filename_xml, use_fbc_package=True)
cobra.io.save_json_model(model, target_filename_json,pretty = True)

In [13]:
model.genes.MCA1740.reactions

frozenset({<Reaction ITCOALm at 0x114b29590>,
           <Reaction SUCOAS at 0x114b5c910>})

In [24]:
model.reactions.EX_co2_e.lower_bound = -1000

In [37]:
model.reactions.GAPDH_nadp.upper_bound = 0

In [36]:
model.reactions.GAPD.lower_bound = -1000

In [34]:
model.reactions.PGK.lower_bound = -1000

In [65]:
model.reactions.AKGDH

Id,AKGDH
Name,2-Oxogluterate dehydrogenase
Stoichiometry,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
GPR,MCA1952 and MCA1953
Lower bound,0.000000
Upper bound,0.000000


In [64]:
model.reactions.OOR3r

Id,OOR3r
Name,2-oxoglutarate synthase (rev)
Stoichiometry,co2_c + fdxrd_c + succoa_c --> akg_c + coa_c + fdxox_c + h_c
GPR,MCA1952 and MCA1953
Lower bound,0.000000
Upper bound,0.000000


In [26]:
model.reactions.TKT2.gene_reaction_rule = 'MCA3040 or MCA3046'

In [27]:
model.reactions.PFK_3_ppi.gene_reaction_rule = model.reactions.PFK_ppi.gene_reaction_rule

In [47]:
model.reactions.EX_ch4_e.lower_bound = 0

In [52]:
model.reactions.TPI

Id,TPI
Name,Triose-phosphate isomerase
Stoichiometry,dhap_c <=> g3p_c
GPR,MCA0674
Lower bound,-1000.000000
Upper bound,1000.000000


In [49]:
model.reactions.EX_co2_e.flux

1.2378688524590173

In [48]:
model.solve().f

Infeasible: Solving model BMID000000141026 did not return an optimal solution. The returned solution status is "infeasible"

In [86]:
model.reactions.G6PDH2

Id,G6PDH2
Name,"1 H(+) + 1 6-O-phosphono-D-glucono-1,5-lactone + 1 NADPH = 1 NADP(+) + 1 beta-D-glucose 6-phosphate(2-)"
Stoichiometry,g6p_c + nadp_c --> 6pgl_c + h_c + nadph_c
GPR,MCA0025 or MCA2971
Lower bound,0.000000
Upper bound,1000.000000


In [6]:
[met for met in model.metabolites if met.id.endswith('_om')]

[]

In [3]:
len(model.reactions)

892

In [4]:
len(model.metabolites)

883

In [5]:
ecoli = load_model('iJO1366',sanitize=False)

In [6]:
len(ecoli.reactions)

2583

In [7]:
def find_demand_and_exchange_reactions(model):
    """
    Return a list containing demand and exchange reactions of model.

    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    """
    return [rxn for rxn in model.reactions if len(rxn.metabolites.keys()) == 1]

In [8]:
def find_transport_reactions(model):
    """
    Return a list of all transport reactions.

       A transport reaction is defined as follows:
       -- It contains metabolites from at least 2 compartments
       -- At least 1 metabolite undergoes no chemical reaction
          i.e. the formula stays the same on both sides of the equation

       Will not identify transport via the PTS System.

    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    """
    compartment_spanning_rxns = \
        [rxn for rxn in model.reactions if len(rxn.get_compartments()) >= 2]

    transport_reactions = []
    for rxn in compartment_spanning_rxns:
        rxn_reactants = set([met.formula for met in rxn.reactants])
        rxn_products = set([met.formula for met in rxn.products])

        transported_mets = \
            [formula for formula in rxn_reactants if formula in rxn_products]
        # Excluding H-pumping reactions for now.
        if set(transported_mets).issubset(set('H')):
            pass
        # Excluding redox-reactions which only transport electrons
        elif set(transported_mets).issubset(set(['X', 'XH2'])):
            pass

        elif len(transported_mets) >= 1:
            transport_reactions.append(rxn)

    return transport_reactions

In [9]:
def find_biomass_reaction(model):
    """
    Return a list of the biomass reaction(s) of the model.

    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    """
    return [rxn for rxn in model.reactions if "biomass" in rxn.id.lower()]

In [10]:
exchanges = find_demand_and_exchange_reactions(model)
biomass = find_biomass_reaction(model)
total_rxns = model.reactions
metab_rxns = set(total_rxns).difference(set().union(exchanges, biomass))

In [11]:
len(find_transport_reactions(model))

120

In [12]:
len(exchanges)

51

In [13]:
len(metab_rxns)

840

In [1]:
set([met[:-2] for met in model.metabolites])

NameError: name 'model' is not defined

In [14]:
unique_mets = []
for met in model.metabolites:
    if met.id[:-2] not in unique_mets:
        unique_mets.append(met.id[:-2])

In [15]:
len(unique_mets)

783

In [16]:
[met for met in model.metabolites if met.id.endswith('_im')]

[<Metabolite q8_im at 0x115b148d0>, <Metabolite q8h2_im at 0x115b149d0>]

In [17]:
model.compartments.pop('om')

u'outer membrane'

In [18]:
model.compartments.pop('bm')

u'biomass'

In [19]:
model.compartments

{u'c': u'cytosol',
 u'e': u'extracellular',
 u'im': u'inner membrane',
 u'p': u'periplasm'}

In [20]:
len(model.genes)

730

In [21]:
len([rxn for rxn in model.reactions if 'metanetx.reaction' in rxn.annotation])

706

In [22]:
len(model.metabolites)

883

In [23]:
len([met for met in model.metabolites if 'metanetx.chemical' in met.annotation])

829

In [24]:
model.reactions.PMMOipp.notes

{u'COFACTOR': [u'2 Cu2+', u'1-2 Fe'],
 u'CONFIDENCE SCORE': u'4',
 u'INHIBITOR': u'',
 u'LOCALIZATION': u'Inner Membrane'}

In [25]:
enzyme_complexes = []
for rxn in model.reactions:
    if rxn.gene_reaction_rule != '' and 'and' in rxn.gene_reaction_rule \
    and rxn.gene_reaction_rule not in enzyme_complexes:
        enzyme_complexes.append(rxn.gene_reaction_rule)
        

In [26]:
len(enzyme_complexes)

24

In [27]:
enzyme_complexes

[u'(MCA0967 and MCA0968) or (MCA1740 and MCA1741)',
 u'((MCA2576 or MCA1391 or MCA1210) and (MCA1393 or MCA1208) and (MCA1209 or MCA1392) and MCA1390 and MCA1389)',
 u'( MCA1194 and MCA1195 and MCA1198 and MCA1196 and MCA1200 and MCA1202 and MCA1205 )',
 u'MCA1952 and MCA1953',
 u'(((MCA1796 and MCA1797 and MCA1798) or (MCA2853 and MCA2854 and MCA2855)) and MCA0295)',
 u'(((( MCA0299 and MCA0300 ) and (MCA1528 and MCA1529 and MCA1530 and MCA1525 and MCA1527)) or (MCA0779 and MCA0780 and MCA0786 and MCA0787 and MCA0788 and MCA0783 and MCA0785)) and MCA0782 and MCA0781 and (MCA0789 or MCA2381))',
 u'( MCA1960_c and MCA1961_c and MCA1962 and MCA2405)',
 u'( MCA2618 and MCA2405)',
 u'(MCA1185 and MCA1186 and MCA1187)',
 u'((( MCA0879 and MCA0880) or (MCA2396 and MCA2397)) and MCA0882 and MCA0883)',
 u'((MCA0005 and MCA0006 and MCA0007 and MCA0008 and MCA0009 and MCA0010 and MCA0011 and MCA0012 and MCA0013) or (MCA2699 and MCA2700 and MCA2701 and MCA2707 and MCA2708 and MCA1556 and MCA3021)

In [8]:
import escher
        
maps = escher.Builder(
                    model=model,
                    map_name = 'Methylococcus capsulatus genome-scale metabolic model',
                    map_json='/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/18RXNplacedInSubsystems.json',
                    show_gene_reaction_rules=True,
                    use_3d_transform = True,
                    )

maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
   { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
   { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
maps.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]

stopping Server...


127.0.0.1 - - [16/May/2017 19:30:12] "GET / HTTP/1.1" 200 -


In [29]:
import json

In [63]:
import re

In [186]:
with open('/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/16UpdateDuplicateAndParallelReactions.json','r') as data_file:
    data = json.load(data_file) 
    
    for rxn in data[1]['reactions']:
        data[1]['reactions'][rxn]['bigg_id'] = data[1]['reactions'][rxn]['bigg_id'][:-2]
        #map_rxn_mets = {met['bigg_id'] + ': ' + str(float(met['coefficient'])) for met in data[1]['reactions'][rxn]['metabolites']}
        map_rxn_mets = {met['bigg_id'] for met in data[1]['reactions'][rxn]['metabolites']}
        for reaction in model.reactions:
            #model_rxn_mets = {key.id + ': ' + str(value) for (key,value) in reaction.metabolites.items()}
            model_rxn_mets = {key.id for (key,value) in reaction.metabolites.items()}
            core_pattern = re.split('([A-Z0-9]+)', reaction.id, maxsplit=1)
            if map_rxn_mets == model_rxn_mets and core_pattern[1] in data[1]['reactions'][rxn]['bigg_id']:
                data[1]['reactions'][rxn]['bigg_id'] = reaction.id
                data[1]['reactions'][rxn]['name'] = reaction.name
                data[1]['reactions'][rxn]['reversibility'] = reaction.reversibility
            else:
                pass
            
    unmapped_ids = []
    for rxn in data[1]['reactions']:
        if data[1]['reactions'][rxn]['bigg_id'] not in model.reactions:
            unmapped_ids.append(data[1]['reactions'][rxn]['bigg_id'] + rxn)
    
with open('/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/17RemappingOfIDs.json', 'w') as f:
    json.dump(data, f)

In [187]:
len(unmapped_ids)

54

In [3]:
solution = model.solve()

In [191]:
model.metabolites.hac.reactions

frozenset({<Reaction HCITS at 0x1159a0550>})

In [ ]:
import escher
        
maps = escher.Builder(
                    model=model,
                    map_name = 'Methylococcus capsulatus genome-scale metabolic model',
                    map_json='/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/18RXNplacedInSubsystems.json',
                    # reaction_data = solution.fluxes,
                    show_gene_reaction_rules=True,
                    use_3d_transform = True,
                    )

maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
   { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
   { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
maps.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


/Users/clie/.virtualenvs/modeling/lib/python2.7/site-packages/escher/plots.py:356: UserWarning:

map_json overrides map_name

127.0.0.1 - - [05/May/2017 23:59:20] "GET / HTTP/1.1" 200 -


In [201]:
model.reactions.SMMOi.subsystem

u'rn00680:Methane metabolism,rn01100:Metabolic pathways,rn01120:Microbial metabolism in diverse environments,rn01200:Carbon metabolism'

In [202]:
len([rxn for rxn in model.reactions if rxn.subsystem != ''])

644

In [229]:
model.reactions.HEX1.annotation

{u'bigg.reaction': u'HEX1',
 u'kegg.reaction': u'R00299',
 u'metanetx.reaction': u'MNXR2720',
 u'rhea': [u'17825', u'17826', u'17827', u'17828'],
 u'seed': u'rxn00216',
 u'upa': [u'UPA:UCR00299', u'UPA:UER00180']}

In [235]:
len([rxn for rxn in model.reactions if '' in rxn.annotation])

311

In [230]:
len([rxn for rxn in model.reactions if 'metanetx.reaction' in rxn.annotation])

706

In [228]:
len([rxn for rxn in model.reactions if 'kegg.reaction' in rxn.annotation])

634

In [207]:
no_sub = [rxn for rxn in model.reactions if rxn.subsystem == '']

In [ ]:
model.reactions.F

In [218]:
model.reactions.MC_AFAA.id.split('_')[1]

u'AFAA'

In [223]:
subsys_dic = {}

for rxn in model.reactions:
    if rxn.id.startswith('EX_') or rxn.id.startswith('DM_'):
        pass
    elif rxn.id.startswith('MC_'):
        r_id = rxn.id.split('_')[1]
        if r_id in ecoli.reactions:
            subsys_dic[r_id] = ecoli.reactions.get_by_id(r_id).subsystem
    else:
        r_id = rxn.id.split('_')[0]
        if r_id in ecoli.reactions:
            subsys_dic[r_id] = ecoli.reactions.get_by_id(r_id).subsystem

In [226]:
subsys_dic

{u'3HAD100': u'Cell Envelope Biosynthesis',
 u'3HAD120': u'Cell Envelope Biosynthesis',
 u'3HAD140': u'Cell Envelope Biosynthesis',
 u'3HAD160': u'Cell Envelope Biosynthesis',
 u'3HAD180': u'Cell Envelope Biosynthesis',
 u'3HAD40': u'Cell Envelope Biosynthesis',
 u'3HAD60': u'Cell Envelope Biosynthesis',
 u'3HAD80': u'Cell Envelope Biosynthesis',
 u'3OAR100': u'Cell Envelope Biosynthesis',
 u'3OAR120': u'Cell Envelope Biosynthesis',
 u'3OAR140': u'Cell Envelope Biosynthesis',
 u'3OAR160': u'Cell Envelope Biosynthesis',
 u'3OAR180': u'Cell Envelope Biosynthesis',
 u'3OAR40': u'Cell Envelope Biosynthesis',
 u'3OAR60': u'Cell Envelope Biosynthesis',
 u'3OAR80': u'Cell Envelope Biosynthesis',
 u'3OAS100': u'Cell Envelope Biosynthesis',
 u'3OAS120': u'Cell Envelope Biosynthesis',
 u'3OAS140': u'Cell Envelope Biosynthesis',
 u'3OAS160': u'Cell Envelope Biosynthesis',
 u'3OAS180': u'Cell Envelope Biosynthesis',
 u'3OAS60': u'Cell Envelope Biosynthesis',
 u'3OAS80': u'Cell Envelope Biosynthesi

In [211]:
sub_ec = [rxn for rxn in ecoli.reactions if rxn.subsystem != '']

In [209]:
no_sub_ec = [rxn for rxn in ecoli.reactions if rxn.subsystem == '']